In [2]:
import numpy as np


In [3]:
N = 5
a = np.arange(1,N**2 + 1).reshape(N,N)
print(a)
print(a.shape)

[[ 1  2  3  4  5]
 [ 6  7  8  9 10]
 [11 12 13 14 15]
 [16 17 18 19 20]
 [21 22 23 24 25]]
(5, 5)


In [4]:
b = np.stack([a,2*a])
print(b.shape)
b = np.expand_dims(b,0)
print(b.shape)
print(b[0][0])
print(b[0][1])

(2, 5, 5)
(1, 2, 5, 5)
[[ 1  2  3  4  5]
 [ 6  7  8  9 10]
 [11 12 13 14 15]
 [16 17 18 19 20]
 [21 22 23 24 25]]
[[ 2  4  6  8 10]
 [12 14 16 18 20]
 [22 24 26 28 30]
 [32 34 36 38 40]
 [42 44 46 48 50]]


In [5]:
def zero_weave_forward(arr, weave_param):
    """
    input array size = (num_img, num_filters, height, width)
    weave_param = {'num_zeros': num_zeros}
    output array size = (num_img, num_filters, 
                        height*(num_zeros + 1) - num_zeros, 
                        width*(num_zeros + 1) - num_zeros)
    Ex: 
    a = [[ 1  2  3  4  5]
        [ 6  7  8  9 10]
        [11 12 13 14 15]
        [16 17 18 19 20]
        [21 22 23 24 25]]
    >>> b = zero_weave(a, {'num_zeros':2})
    b = [[ 1.  0.  0.  6.  0.  0. 11.  0.  0. 16.  0.  0. 21.]
        [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
        [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
        [ 2.  0.  0.  7.  0.  0. 12.  0.  0. 17.  0.  0. 22.]
        [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
        [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
        [ 3.  0.  0.  8.  0.  0. 13.  0.  0. 18.  0.  0. 23.]
        [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
        [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
        [ 4.  0.  0.  9.  0.  0. 14.  0.  0. 19.  0.  0. 24.]
        [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
        [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
        [ 5.  0.  0. 10.  0.  0. 15.  0.  0. 20.  0.  0. 25.]]
    
    """
    num_zeros = weave_param['num_zeros']
    cache = (arr, weave_param)
    
    num_img, num_filters, height, width = arr.shape
    
    zero_copy = np.zeros([num_img,
                          num_filters,
                          height*(num_zeros + 1) - num_zeros,
                          width*(num_zeros + 1) - num_zeros])
    slice_jump = num_zeros + 1
    zero_copy[:,:,::slice_jump,::slice_jump] = arr
    return zero_copy, cache

def zero_weave_backwards(dout, cache):
    (_, weave_param) = cache
    num_zeros = weave_param['num_zeros']
    slice_jump = num_zeros + 1
    dx = dout[:,:,::slice_jump,::slice_jump]
    return dx

#def generate_weave_multiplier(arr, weave_param):
    
def array_weave_forwards(arr, weave_param):
    """
    input array size = (num_img, num_filters, height, width)
    output array size = (num_img, num_filters, 
                        height*(num_zeros + 1) - num_zeros, 
                        width*(num_zeros + 1) - num_zeros)
    Ex: 
    a = [[ 1  2  3  4  5]
        [ 6  7  8  9 10]
        [11 12 13 14 15]
        [16 17 18 19 20]
        [21 22 23 24 25]]
    >>> b = zero_weave(a, {'num_zeros':2})
    b = [[ 1.  4.  0.  2.  5.  0.  3.  0.  1.  4.  0.  2.  5.]
        [16. 19.  0. 17. 20.  0. 18.  0. 16. 19.  0. 17. 20.]
        [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
        [ 6.  9.  0.  7. 10.  0.  8.  0.  6.  9.  0.  7. 10.]
        [21. 24.  0. 22. 25.  0. 23.  0. 21. 24.  0. 22. 25.]
        [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
        [11. 14.  0. 12. 15.  0. 13.  0. 11. 14.  0. 12. 15.]
        [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
        [ 1.  4.  0.  2.  5.  0.  3.  0.  1.  4.  0.  2.  5.]
        [16. 19.  0. 17. 20.  0. 18.  0. 16. 19.  0. 17. 20.]
        [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
        [ 6.  9.  0.  7. 10.  0.  8.  0.  6.  9.  0.  7. 10.]
        [21. 24.  0. 22. 25.  0. 23.  0. 21. 24.  0. 22. 25.]]

    
    """
    num_zeros = weave_param['num_zeros']
    filter_size = weave_param['filter_size']
    cache = (arr, weave_param)
    
    num_img, num_filters, height, width = arr.shape
    
    zero_copy = np.zeros([num_img,
                      num_filters,
                      height*(num_zeros + 1) - num_zeros,
                      width*(num_zeros + 1) - num_zeros])
    #This needs to be generalized to other filter sizes
    expand_dist = 2*filter_size+2
    slice_jump = num_zeros + 1
    
    for i in range(height):
        for j in range(width):
            temp_val = arr[:,:,i,j]
            big_i = filter_size * i 
            big_j = filter_size * j
            for i_change in [-expand_dist, 0, expand_dist]:
                for j_change in [-expand_dist, 0, expand_dist]:
                    if (min(big_i+i_change,big_j+j_change) >= 0 
                        and max(big_i+i_change,big_j+j_change) < height*(num_zeros + 1) - num_zeros):
                        zero_copy[:,:,big_i+i_change,big_j+j_change] = temp_val

    zero_copy[:,:,::slice_jump,::slice_jump] = 0
                     
    return zero_copy, cache

def array_weave_backwards(dout, cache):
    org_arr, weave_param = cache
    num_zeros = weave_param['num_zeros']
    filter_size = weave_param['filter_size']
    
    expand_dist = 2*filter_size+2
    slice_jump = num_zeros + 1
    
    num_img, num_filters, height, width = org_arr.shape
    
    
    dx = np.zeros([num_img,
                   num_filters,
                   height,
                   width]) 
    for i in range(height):
        for j in range(width):
            big_i = filter_size * i 
            big_j = filter_size * j
            for i_change in [-expand_dist, 0, expand_dist]:
                for j_change in [-expand_dist, 0, expand_dist]:
                    if (min(big_i+i_change,big_j+j_change) >= 0 
                        and max(big_i+i_change,big_j+j_change) < height*(num_zeros + 1) - num_zeros):
                        dx[:,:,i,j] += dout[:,:,big_i+i_change,big_j+j_change]
    return dx

def array_sum_fowards(arr1, arr2):
    cache = (arr1, arr2)
    sum_array = arr1 + arr2
    return sum_array, cache

def array_sum_backwards(dx, cache):
    dout = np.ones_like(dx)
    return dout, dout

    
    

In [6]:
weave_param = {'num_zeros':2, 'filter_size': 3}
c, cache = zero_weave_forward(b, weave_param)
print c
d = zero_weave_backwards(c,cache)
print(d)

[[[[ 1.  0.  0.  2.  0.  0.  3.  0.  0.  4.  0.  0.  5.]
   [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
   [ 6.  0.  0.  7.  0.  0.  8.  0.  0.  9.  0.  0. 10.]
   [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
   [11.  0.  0. 12.  0.  0. 13.  0.  0. 14.  0.  0. 15.]
   [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
   [16.  0.  0. 17.  0.  0. 18.  0.  0. 19.  0.  0. 20.]
   [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
   [21.  0.  0. 22.  0.  0. 23.  0.  0. 24.  0.  0. 25.]]

  [[ 2.  0.  0.  4.  0.  0.  6.  0.  0.  8.  0.  0. 10.]
   [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
   [12.  0.  0. 14.  0.  0. 16.  0.  0. 18.  0.  0. 20.]
   [ 0.  0.  0.  0.  0.  0.  

In [7]:
N = 5

ones = np.ones([N,N])
temp = np.arange(1,N**2 + 1).reshape([N,N])
print(temp)
ones = np.stack([temp,ones])
ones = np.expand_dims(ones,0)

e, cache = array_weave_forwards(ones, weave_param)
print(e)
f = array_weave_backwards(e, cache)
print(f)

[[ 1  2  3  4  5]
 [ 6  7  8  9 10]
 [11 12 13 14 15]
 [16 17 18 19 20]
 [21 22 23 24 25]]
[[[[ 0.  4.  0.  0.  5.  0.  0.  0.  1.  0.  0.  2.  0.]
   [16. 19.  0. 17. 20.  0. 18.  0. 16. 19.  0. 17. 20.]
   [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
   [ 0.  9.  0.  0. 10.  0.  0.  0.  6.  0.  0.  7.  0.]
   [21. 24.  0. 22. 25.  0. 23.  0. 21. 24.  0. 22. 25.]
   [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
   [ 0. 14.  0.  0. 15.  0.  0.  0. 11.  0.  0. 12.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
   [ 1.  4.  0.  2.  5.  0.  3.  0.  1.  4.  0.  2.  5.]
   [ 0. 19.  0.  0. 20.  0.  0.  0. 16.  0.  0. 17.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
   [ 6.  9.  0.  7. 10.  0.  8.  0.  6.  9.  0.  7. 10.]
   [ 0. 24.  0.  0. 25.  0.  0.  0. 21.  0.  0. 22.  0.]]

  [[ 0.  1.  0.  0.  1.  0.  0.  0.  1.  0.  0.  1.  0.]
   [ 1.  1.  0.  1.  1.  0.  1.  0.  1.  1.  0.  1.  1.]
   [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 

In [26]:
print(temp)
temp_weave = e[0][0]
print(temp_weave)
print(temp_weave.shape)
print(np.where([temp == 5])[2])

[[ 1  2  3  4  5]
 [ 6  7  8  9 10]
 [11 12 13 14 15]
 [16 17 18 19 20]
 [21 22 23 24 25]]
[[ 0.  4.  0.  0.  5.  0.  0.  0.  1.  0.  0.  2.  0.]
 [16. 19.  0. 17. 20.  0. 18.  0. 16. 19.  0. 17. 20.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  9.  0.  0. 10.  0.  0.  0.  6.  0.  0.  7.  0.]
 [21. 24.  0. 22. 25.  0. 23.  0. 21. 24.  0. 22. 25.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. 14.  0.  0. 15.  0.  0.  0. 11.  0.  0. 12.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  4.  0.  2.  5.  0.  3.  0.  1.  4.  0.  2.  5.]
 [ 0. 19.  0.  0. 20.  0.  0.  0. 16.  0.  0. 17.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 6.  9.  0.  7. 10.  0.  8.  0.  6.  9.  0.  7. 10.]
 [ 0. 24.  0.  0. 25.  0.  0.  0. 21.  0.  0. 22.  0.]]
(13, 13)
[4]


In [39]:
def create_I_matrix(N, weave_param):
    N = np.arange(1,N**2 + 1).reshape([N,N])
    N_weave,_ = array_weave_forwards(ones, weave_param)
    
    num_img, num_filters, num_rows, num_cols = N_weave.shape
    I = np.zeros([2,num_rows,num_cols])
    for i in range(num_cols):
        for j in range(num_rows):
            i_j_loc = np.where(N_weave[0,0,i,j] == N)
            print(i_j_loc)
            I[:,i,j] = i_j_loc[1:]
    return I
        

In [40]:
print(create_I_matrix(5,weave_param))

[[[[ 0.  4.  0.  0.  5.  0.  0.  0.  1.  0.  0.  2.  0.]
   [16. 19.  0. 17. 20.  0. 18.  0. 16. 19.  0. 17. 20.]
   [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
   [ 0.  9.  0.  0. 10.  0.  0.  0.  6.  0.  0.  7.  0.]
   [21. 24.  0. 22. 25.  0. 23.  0. 21. 24.  0. 22. 25.]
   [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
   [ 0. 14.  0.  0. 15.  0.  0.  0. 11.  0.  0. 12.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
   [ 1.  4.  0.  2.  5.  0.  3.  0.  1.  4.  0.  2.  5.]
   [ 0. 19.  0.  0. 20.  0.  0.  0. 16.  0.  0. 17.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
   [ 6.  9.  0.  7. 10.  0.  8.  0.  6.  9.  0.  7. 10.]
   [ 0. 24.  0.  0. 25.  0.  0.  0. 21.  0.  0. 22.  0.]]

  [[ 0.  1.  0.  0.  1.  0.  0.  0.  1.  0.  0.  1.  0.]
   [ 1.  1.  0.  1.  1.  0.  1.  0.  1.  1.  0.  1.  1.]
   [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
   [ 0.  1.  0.  0.  1.  0.  0.  0.  1.  0.  0.  1.  0.]
   [ 1.  1.  0.  1.  1.  0.  

ValueError: setting an array element with a sequence.